# Step 2 - Database Engineering

Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.

Use Pandas to read your cleaned measurements and stations CSV data.

Use the engine and connection string to create a database called hawaii.sqlite.

Use declarative_base and create ORM classes for each table.

You will need a class for Measurement and for Station.

Make sure to define your primary keys.

Once you have your ORM classes defined, create the tables in the database using create_all.

In [68]:
# Python SQL toolkit and Object Relational Mapper
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [69]:
!rm hawaii.sqlite

# Use the engine and connection string to create a database called hawaii.sqlite.
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [70]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    hw_date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

class Station(Base):
    __tablename__ = 'station'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [71]:
# Use `create_all` to create the table in the database
Base.metadata.create_all(engine)

engine.table_names()

['measurement', 'station']

In [74]:
clean_df = pd.read_csv('clean_hawaii_measurements.csv')
#clean_df['hw_date'] =  pd.to_datetime(clean_df['hw_date'])

measurements_clean_df = clean_df.to_dict(orient='records')

pd_hawaii_stations = pd.read_csv("hawaii_stations.csv")
stations_df = pd_hawaii_stations.to_dict(orient='records')


clean_df.dtypes

Unnamed: 0      int64
station        object
hw_date        object
prcp          float64
tobs            int64
dtype: object

In [75]:
print(measurements_clean_df[:5])

[{'Unnamed: 0': 0, 'station': 'USC00519397', 'hw_date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'Unnamed: 0': 1, 'station': 'USC00519397', 'hw_date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'Unnamed: 0': 2, 'station': 'USC00519397', 'hw_date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'Unnamed: 0': 3, 'station': 'USC00519397', 'hw_date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'Unnamed: 0': 4, 'station': 'USC00519397', 'hw_date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]


In [76]:
print(stations_df[:5])

[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'elevation': 306.6}]


In [77]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [78]:
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [79]:
conn.execute(measurement_table.delete())
conn.execute(station_table.delete())

In [80]:
conn.execute(station_table.insert(), stations_df)
conn.execute(measurement_table.insert(), measurements_clean_df)

In [81]:
conn.execute("select * from measurement limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0),
 (6, 'USC00519397', '2010-01-08', 0.0, 64.0),
 (7, 'USC00519397', '2010-01-09', 0.0, 68.0),
 (8, 'USC00519397', '2010-01-10', 0.0, 73.0),
 (9, 'USC00519397', '2010-01-11', 0.01, 64.0),
 (10, 'USC00519397', '2010-01-12', 0.0, 61.0)]

In [82]:
conn.execute("select * from station limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]